In [205]:
import pandas as pd
import numpy as np

In [206]:
data = pd.read_excel("URINELOTS.xlsx")
#data.head()

In [207]:
data.shape

(146, 15)

In [163]:
#data.head()

#### DATA cleaning

In [211]:
data = data.dropna(subset=['RAMJA Results','AIIMS Result culture'])

In [213]:
scores = data.drop(['RAMJA Results','AIIMS Result culture'], axis=1)
scores.insert(0,'UHID', scores.pop('UHID'))
scores.head()

,UHID,Patients Name,Age,Gender,Date,E.coli,K. Pneumoniae,P.aurogenosa,A.baumnnii,P.mirabilis,E.faecalis,S. aureus,S. saprophyticus
0,20220065432,Sahroj,33.0,F,13/02/2024,NaN,NaN,-5.26086,-4.09794,-3.58480,-8.10731,-5.62819,NaN
2,20230125389,Saloni,19.0,F,15/02/2024,-15.30980,-10.01740,-9.09993,-8.30414,-5.06676,-5.42121,-8.77797,NaN
3,20180022685,Ravita,34.0,F,15/02/2024,-7.70704,-8.08910,-4.79176,-8.63416,-11.84570,-9.31657,-9.90339,
5,20240012395,kedari Devi,38.0,F,15/02/2024,-10.25510,-7.26028,-6.89303,-14.34320,-11.54470,-10.78790,-10.82420,NaN
6,20220166372,Babita Devi,45.0,F,16/02/2024,-8.87334,-5.39780,-12.27840,-8.01114,-11.00330,-13.59960,-11.66680,NaN


In [215]:
import re
def cleanup(text):
    text = text.upper()
    text = text.replace(".", " ")
    text = text.replace(" AND ", " ")
    text = re.sub(r'\s+', ' ', text)  # Use re.sub for regular expression replacement
    text = text.strip()
    return text


In [216]:
bacteriatypes = {
    "COLI" : 'Escherichia coli', 
    "PNEUMO" : 'Klebsiella pneumoniae',
    "AERUGI" : 'Pseudomonas aeruginosa',
    "BAUMAN" : 'Acinetobacter baumannii', 
    "MIRABIL" : 'Proteus mirabilis', 
    "FAECALI" : 'Enterococcus faecalis', 
    "AUREUS" : 'Staphylococcus aureus', 
    "SAPROPHY" : 'Staphylococcus saprophyticus',
    "STERI" : 'Sterile'
}
#['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']

#### RAMJA results

In [217]:
resultsRAMJA = data[['UHID','RAMJA Results']]

In [218]:
resultsRAMJA['cleanRAMJA'] = resultsRAMJA['RAMJA Results'].apply(cleanup)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\766998183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA['cleanRAMJA'] = resultsRAMJA['RAMJA Results'].apply(cleanup)


In [219]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI..."
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA..."
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI..."
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS"


In [220]:
def check(mainstring, s1, s2):
    if (mainstring.count(s1) > 0 or mainstring.count(s2) > 0):
        #print("YES")
        return(1)
    else:
        #print("NO")
        return(0)


mainstring = "E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI"
s1 = "E COLI"
s2 = "K PNEUMONIAE"
check(mainstring, s2, s1)

1

In [221]:
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI"]
for s1, s2 in zip(genus,species):
    resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [182]:
#resultsRAMJA.head()

In [222]:
resultsRAMJA.rename(columns=bacteriatypes, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\78547064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA.rename(columns=bacteriatypes, inplace=True)


In [184]:
#resultsRAMJA.head()

In [223]:
columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']

In [224]:
resultsRAMJA.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\1583815861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)


In [225]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA,Escherichia coli_R,Klebsiella pneumoniae_R,Pseudomonas aeruginosa_R,Acinetobacter baumannii_R,Proteus mirabilis_R,Enterococcus faecalis_R,Staphylococcus aureus_R,Staphylococcus saprophyticus_R,Sterile_R
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,0,0
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,0,0
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,0,0


In [226]:
resultsRAMJA.shape

(118, 12)

#### AIIMS results

In [227]:
resultsAIIMS = data[['UHID','AIIMS Result culture']]

In [228]:
resultsAIIMS['cleanAIIMS'] = resultsAIIMS['AIIMS Result culture'].apply(cleanup)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\265187514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS['cleanAIIMS'] = resultsAIIMS['AIIMS Result culture'].apply(cleanup)


In [229]:
#resultsAIIMS.head()

In [230]:
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI"]
for s1, s2 in zip(genus,species):
    resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\2996149377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\2996149377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\2996149377.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [192]:
#resultsAIIMS.head()

In [231]:
resultsAIIMS.rename(columns=bacteriatypes, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\4113831117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS.rename(columns=bacteriatypes, inplace=True)


In [194]:
#resultsAIIMS.head()

In [232]:
resultsAIIMS.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9308\3656104579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)


In [233]:
resultsAIIMS.head()

,UHID,AIIMS Result culture,cleanAIIMS,Escherichia coli_A,Klebsiella pneumoniae_A,Pseudomonas aeruginosa_A,Acinetobacter baumannii_A,Proteus mirabilis_A,Enterococcus faecalis_A,Staphylococcus aureus_A,Staphylococcus saprophyticus_A,Sterile_A
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,E.coli,E COLI,1,0,0,0,0,0,0,0,0
3,20180022685,CONTAMINATION,CONTAMINATION,0,0,0,0,0,0,0,0,0
5,20240012395,E.coli,E COLI,1,0,0,0,0,0,0,0,0
6,20220166372,E.coli & Pseudomnas,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


In [234]:
resultsAIIMS.shape

(118, 12)

#### Combining both AIIMS and RAMJA results

In [252]:
results = pd.merge(resultsRAMJA, resultsAIIMS, on="UHID", how='inner')

In [253]:
results.head()

,UHID,RAMJA Results,cleanRAMJA,Escherichia coli_R,Klebsiella pneumoniae_R,Pseudomonas aeruginosa_R,Acinetobacter baumannii_R,Proteus mirabilis_R,Enterococcus faecalis_R,Staphylococcus aureus_R,...,cleanAIIMS,Escherichia coli_A,Klebsiella pneumoniae_A,Pseudomonas aeruginosa_A,Acinetobacter baumannii_A,Proteus mirabilis_A,Enterococcus faecalis_A,Staphylococcus aureus_A,Staphylococcus saprophyticus_A,Sterile_A
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,...,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
1,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,...,E COLI,1,0,0,0,0,0,0,0,0
2,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,...,CONTAMINATION,0,0,0,0,0,0,0,0,0
3,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,...,E COLI,1,0,0,0,0,0,0,0,0
4,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,...,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


In [254]:
data.shape

(118, 15)

In [255]:
results.shape

(120, 23)

In [257]:
UrineLot1 = pd.merge(scores, results, on = 'UHID', how='inner')

In [258]:
UrineLot1.shape

(124, 35)

In [259]:
UrineLot1.head()

,UHID,Patients Name,Age,Gender,Date,E.coli,K. Pneumoniae,P.aurogenosa,A.baumnnii,P.mirabilis,...,cleanAIIMS,Escherichia coli_A,Klebsiella pneumoniae_A,Pseudomonas aeruginosa_A,Acinetobacter baumannii_A,Proteus mirabilis_A,Enterococcus faecalis_A,Staphylococcus aureus_A,Staphylococcus saprophyticus_A,Sterile_A
0,20220065432,Sahroj,33.0,F,13/02/2024,NaN,NaN,-5.26086,-4.09794,-3.58480,...,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
1,20230125389,Saloni,19.0,F,15/02/2024,-15.30980,-10.01740,-9.09993,-8.30414,-5.06676,...,E COLI,1,0,0,0,0,0,0,0,0
2,20180022685,Ravita,34.0,F,15/02/2024,-7.70704,-8.08910,-4.79176,-8.63416,-11.84570,...,CONTAMINATION,0,0,0,0,0,0,0,0,0
3,20240012395,kedari Devi,38.0,F,15/02/2024,-10.25510,-7.26028,-6.89303,-14.34320,-11.54470,...,E COLI,1,0,0,0,0,0,0,0,0
4,20220166372,Babita Devi,45.0,F,16/02/2024,-8.87334,-5.39780,-12.27840,-8.01114,-11.00330,...,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


In [260]:
#results.to_excel("URINELOT1.xlsx")

#### Compare few results

In [121]:
print("RAMJA : \n", resultsRAMJA["Escherichia coli_R"].value_counts())
print("AIIMS : \n",resultsAIIMS["Escherichia coli_A"].value_counts())

RAMJA : 
 0    75
1    43
Name: Escherichia coli_R, dtype: int64
AIIMS : 
 0    100
1     18
Name: Escherichia coli_A, dtype: int64
